In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
!pip install pm4py==2.7.5.2
!pip install tensorflow
!pip install pydot
!pip install graphviz

  Using cached pm4py-2.7.5.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pm4py
    Found existing installation: pm4py 2.1.4.1
    Uninstalling pm4py-2.1.4.1:
      Successfully uninstalled pm4py-2.1.4.1



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os

In [4]:
#Changing the directory
#os.chdir('/content/drive/My Drive/Process mining course/Part 2 Project')

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import sklearn
import pm4py
import tensorflow


from pydream.predictive.nap.NAP import NAP, multiclass_roc_auc_score
import pickle
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support,precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler

from pydream.EnhancedPN import EnhancedPN
from pm4py.objects.petri_net.importer import importer as pnml_importer
from backend.stat.ci_auc import calculate_auc_ci

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
train_json = "tss_heart_log_train0.json"
valid_json ="tss_heart_log_val0.json"
test_json = "tss_heart_log_test0.json"

In [ ]:
train_algo = NAP(tss_train_file=train_json, tss_test_file=valid_json, options={"n_epochs" : 2})
train_algo.train(checkpoint_path="chk_points", name="HF_model", save_results=True)

Xtrain shape: (8403, 1920)
X2train shape: (8403, 3)
Severity train shape: (8403, 58)
X2test shape: (1531, 3)
Xtest shape: (1531, 1920)
Severity test shape: (1531, 58)
Ytrain shape: (8403, 2)
Ytest shape: (1531, 2)
Instructions for updating:
Colocations handled automatically by placer.


C:\Users\LongVu\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Class weights :  <module 'sklearn.utils.class_weight' from 'C:\\Users\\LongVu\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\sklearn\\utils\\class_weight.py'>
Train on 8403 samples, validate on 1531 samples
Epoch 1/2
1110/8403 [==>...........................] - ETA: 6s - loss: 1.2196 - acc: 0.2928

In [ ]:
algo = NAP()
algo.loadModel(path="chk_points", name="HF_model")

X_test, X2_test, severity_test, Y_test = algo.loadData(test_json)

In [ ]:
stdScaler = MinMaxScaler()
stdScaler.fit(X_test)
X_test = stdScaler.transform(X_test)
stdScaler.fit(X2_test)
X2_test = stdScaler.transform(X2_test)
stdScaler.fit(severity_test)
severity_test = stdScaler.transform(severity_test)

Y_test_int = np.array([int(i) for i in Y_test])
y_prob = algo.model.predict([X_test,X2_test,severity_test])
y_pred = np.argmax(y_prob, axis=1)

test_acc = accuracy_score(Y_test_int, y_pred, normalize=True)

In [ ]:
y_true =Y_test_int
y_probscore = []
list_of_lists = []
for x in y_prob:
    list_of_lists.append(list(x))
    y_probscore.append(list(x)[1])


y_score = np.array(y_probscore)
y_pred = np.around(y_pred)
y_true = np.array(y_true)
auc, auc_cov, ci = calculate_auc_ci(y_true,y_score,y_pred, alpha=0.95)
print("AUC Score: ",auc)
print("AUC_cov: ", auc_cov)
print("confidence interval: ",ci)